# Assignment 1

Github repo for assignment: https://github.com/brentonjackson/csc-4980/tree/master/Assignment1

I'll be using Python for the assignments in this class, as opposed to Matlab.

## Part A: Fundamentals

Go over camera calibration toolbox and calibrate camera.

It may be worth mentioning that in the [DepthAI documentation](https://docs.luxonis.com/projects/hardware/en/latest/pages/guides/calibration.html), for the nonmodular cameras, they've already been calibrated before shipment so recalibration isn't needed. 

However, I've calibrated the camera by following directions at this link as a learning exercise: https://docs.opencv.org/4.x/dc/dbb/tutorial_py_calibration.html

Below is the code I used to do this, in Python (skipped in slideshow), and before and after images:

In [3]:
#!/usr/bin/env python3

# Camera calibration for the OAK-D Lite camera (or any camera)


import numpy as np
import cv2 as cv
import glob

# 1. get object points and image points
# termination criteria
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*7, 3), np.float32)
objp[:, :2] = np.mgrid[0:7, 0:6].T.reshape(-1, 2)
# Arrays to store object points and image points from all the images.
objpoints = []  # 3d point in real world space
imgpoints = []  # 2d points in image plane.
images = glob.glob('../opencv-samples/left*.jpg')
for fname in images:
    img = cv.imread(fname)
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    # Find the chess board corners
    ret, corners = cv.findChessboardCorners(gray, (7, 6), None)
    # If found, add object points, image points (after refining them)
    if ret == True:
        objpoints.append(objp)
        corners2 = cv.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria)
        imgpoints.append(corners2)
        # Draw and display the corners
        cv.drawChessboardCorners(img, (7, 6), corners2, ret)
        cv.imshow('img', img)
        cv.waitKey(500)
cv.destroyAllWindows()


# 2. calibrate camera
ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

# 3. undistort image
img = cv.imread('../opencv-samples/left12.jpg')
h,  w = img.shape[:2]
newcameramtx, roi = cv.getOptimalNewCameraMatrix(mtx, dist, (w,h), 1, (w,h))

# undistort
dst = cv.undistort(img, mtx, dist, None, newcameramtx)
# crop the image
x, y, w, h = roi
dst = dst[y:y+h, x:x+w]
cv.imwrite('calibresult.png', dst)
cv.imshow('calibrate result', dst)
cv.waitKey(500)

-1

Distorted image:

![Distorted image](https://github.com/brentonjackson/csc-4980/blob/master/opencv-samples/left12.jpg?raw=true)

Image after calibration and undistortion:

![Undistorted image](calibresult.png)

As you can see from the curved lines in the first image, there was a considerable amount of radial distortion present.

That was fixed in the latter image.

## Part B: Matlab/Python Prototyping 

Write a MATLAB/Python script to find the real world dimensions (e.g. diameter of a ball, side length of a cube) of an object using perspective projection equations. 

Validate using an experiment where you image an object using your camera from a specific distance (choose any distance but ensure you are able to measure it accurately) between the object and camera.

This assignment requires some background to understand before implementing in code.

## Perspective Projection

### Background

In this example, I use the pinhole camera model to understand perspective projection.

In the previous example, we learned that camera calibration required a few things:
- **Extrinsic parameters** of the camera, e.g. rotation and translation vectors, which translates a coordinate of a 3D point to a coordinate system
- **Intrinsic parameters** of the camera, e.g. focal length and optical centers (both given in the form of a camera matrix) - visit [this link](https://ksimek.github.io/2013/08/13/intrinsic/) for a great breakdown on the intrinsic params
- **Distortion coefficients**

We used images of a chess board to find the camera matrix values since we knew the relative positions of the square corners on the board. By doing this, we were able to find intrinsic parameters of the OAK-D Lite camera.

We can use those parameters (e.g. focal point and optical centers) to undistort any image taken with the camera.

As will be explained in a second, we can also use one of the parameters to help us find real world coordinates of our object from 2d image coordinates. That's the goal of using perspective projection equations.


### Use of Perspective Projection Equations

The general play-by-play of part B will be to:
1. Undistort our image
2. Find some desired dimension in our 2D image, like height or width
3. Use perspective projection equations to convert our desired dimension to 3D, real world values and units

We know how to do 1. We've already done it, so we can modify our camera calibration script from Part A to accept any image, undistort it, then write that new image to the disk for us.

For 2, we can go about it in two ways:
1. Calculate the 2D dimensions of the object(s) in our image using object outlines, bounding boxes, and calculated Euclidean distances of those bounding box sides
2. Allow the user to specify two points of interest and use those points to calculate the 2D dimensions of interest

I will opt for method 2, since it's very easy to test in the real-world.

For 3, this is where we actually make use of the equations which I will go over in the next section.

### Perspective Projection Equations

The perspective projection equations are equations that allow us to convert coordinates on the image plane to coordinates in the real world, and vice-versa. It uses the concept of similar triangles to essentially create a ratio. It's better explained with an image:

![Perspective Projection image](perspective_projection.png)

We can treat the middle plane as our camera and the image plane on the left as the computer screen our image is rendered on.

We want to convert the point P_i on the image plane to the point P_0 in the real world on the right. To do this, we see that the optical axis sets up similar triangles. 

From these equations, we can see that **a point in the real world depends on the ratio of the object distance from the camera to the focal length, times the corresponding coordinate on the image plane**. Since we know the 2D image coordinates and the focal length of the camera, which we got from our camera matrix, we only need to know the distance of the object from the camera.

We can give that distance to our program as a required parameter.

Below is the code that achieves this: